In [13]:
from bs4 import BeautifulSoup
import configparser
import requests
import pandas as pd
import pickle
from constants_funcs import get_session, int_or_float_or_str
import json
import numpy as np
import time
with open('constants.json', 'r') as d:
    constants = json.load(d)

In [2]:
sess = get_session()
sess

creating new login session...


In [40]:
def rundle_lambda(rundle):
    start_idx = int(rundle.text.startswith('Rundle'))
    ini = rundle.text.split()[start_idx:]
    if len(ini) == 1:
        # if ini[0].startswith('Champ'):
        #     return ['A', ini[0], 0]
        # else:
        return [ini[0], 'League', 0]
    elif len(ini) == 2:
        ini.append(0)
    else:
        del ini[2]
        ini[2] = int(ini[2])
    return ini
def get_rundles_by_season(season):
    homepage = sess.get(constants['LLHEADER'])
    html_text = homepage.text
    soup = BeautifulSoup(html_text, 'html.parser')
    cur_season = int(soup.find('nav', {'id': 'menu'}).find('ul').find_all('li')[1].text.split()[0][2:])
    if season < 30 or season > cur_season:
        print(f'season out of range. Valid seasons include from 30 to {cur_season}')
        return None
    if season <= 51:
        url = f'{constants["LLHEADER"]}/ll{season}/questions/md01.shtml'
        standings_page = sess.get(url)
        html_text = standings_page.text
        soup = BeautifulSoup(html_text, 'html.parser')
    if season <= 38: #LL30-38
        return list(map(rundle_lambda, soup.find_all('div', {'class': 'Scorehead'})[:-1]))
    elif season <= 51: #LL39-51
        txt = list(map(lambda h: h.find('a')['href'][24:-13].split('_'), soup.find('ul', {'id': 'resultstabs'}).find_all('li')[1:]))
        rundles = list(map(lambda t:
            [t[0], 'League', 0] if len(t) == 1 else [t[0], t[1], 0], txt))
        return rundles
    elif season < 60: # old format
        url = f'{constants["LLHEADER"]}/standings.php?{season}'
        standings_page = sess.get(url)
        html_text = standings_page.text
        soup = BeautifulSoup(html_text, 'html.parser')
        rundles = list(map(
            lambda tr: rundle_lambda(tr.find('td')),
            soup.find('div', {'class': 'fdr'}).find('table').find_all('tr')[1:]))
        return rundles
    elif season == 62: # thorsten fucked up
        return [
            ['A', 'Central', 0], ['B', 'Central', 0], ['C', 'Central', 0], ['D', 'Central', 1], ['D', 'Central', 2],
            ['E', 'Central', 1], ['E', 'Central', 2], ['R', 'Central', 1], ['R', 'Central', 2],
            ['A', 'Coastal', 0], ['B', 'Coastal', 0], ['C', 'Coastal', 0], ['D', 'Coastal', 1], ['D', 'Coastal', 2],
            ['E', 'Coastal', 1], ['E', 'Coastal', 2], ['R', 'Coastal', 0],
            ['A', 'Corridor', 0], ['B', 'Corridor', 0], ['C', 'Corridor', 0], ['D', 'Corridor', 1], ['D', 'Corridor', 2],
            ['E', 'Corridor', 1], ['E', 'Corridor', 2], ['R', 'Corridor', 1], ['R', 'Corridor', 2],
            ['A', 'Frontier', 0], ['B', 'Frontier', 0], ['C', 'Frontier', 1], ['C', 'Frontier', 2], ['D', 'Frontier', 1],
            ['D', 'Frontier', 2], ['E', 'Frontier', 1], ['E', 'Frontier', 2], ['R', 'Frontier', 1], ['R', 'Frontier', 2],
            ['A', 'Highland', 0], ['B', 'Highland', 0], ['C', 'Highland', 0], ['D', 'Highland', 1], ['D', 'Highland', 2],
            ['E', 'Highland', 1], ['E', 'Highland', 2], ['R', 'Highland', 0],
            ['A', 'Metro', 0], ['B', 'Metro', 0], ['C', 'Metro', 0], ['D', 'Metro', 1], ['D', 'Metro', 2],
            ['E', 'Metro', 1], ['E', 'Metro', 2], ['R', 'Metro', 1], ['R', 'Metro', 2], ['R', 'Metro', 3],
            ['A', 'Midland', 0], ['B', 'Midland', 0], ['C', 'Midland', 0], ['D', 'Midland', 1], ['D', 'Midland', 2],
            ['E', 'Midland', 1], ['E', 'Midland', 2], ['R', 'Midland', 1], ['R', 'Midland', 2],
            ['A', 'Pacific', 0], ['B', 'Pacific', 0], ['C', 'Pacific', 0], ['D', 'Pacific', 1], ['D', 'Pacific', 2],
            ['E', 'Pacific', 1], ['E', 'Pacific', 2], ['R', 'Pacific', 1], ['R', 'Pacific', 2],
            ['A', 'Seaboard', 0], ['B', 'Seaboard', 0], ['C', 'Seaboard', 0], ['D', 'Seaboard', 1], ['D', 'Seaboard', 2],
            ['E', 'Seaboard', 1], ['E', 'Seaboard', 2], ['R', 'Seaboard', 1], ['R', 'Seaboard', 2],
            ['A', 'Skyline', 0], ['B', 'Skyline', 0], ['C', 'Skyline', 0], ['D', 'Skyline', 1], ['D', 'Skyline', 2],
            ['E', 'Skyline', 1], ['E', 'Skyline', 2], ['R', 'Skyline', 0]
        ]
    else:
        url = f'{constants["LLHEADER"]}/allrundles.php?{season}'
        standings_page = sess.get(url)
        html_text = standings_page.text
        soup = BeautifulSoup(html_text, 'html.parser')
        table_index = -1 - int(season >= 82)
        rundles = list(map(rundle_lambda, soup.find_all('table')[table_index].find_all('a')))
        return rundles

def get_all_season_rundles():
    homepage = sess.get(constants['LLHEADER'])
    html_text = homepage.text
    soup = BeautifulSoup(html_text, 'html.parser')
    cur_season = int(soup.find('nav', {'id': 'menu'}).find('ul').find_all('li')[1].text.split()[0][2:])
    leagues_by_season = dict()
    for season in range(30, cur_season + 1):
        result = get_rundles_by_season(season)
        result_leagues = list(set(map(lambda rundle: rundle[1], result)))
        leagues_by_season[season] = {
            'leagues': result_leagues,
            'rundles': result
        }
        print(f'{len(result_leagues)} leagues, {len(result)} rundles in LL{season}')
    return leagues_by_season

all_rundles_by_season = get_all_season_rundles()
all_rundles_by_season[31]['rundles'] = [i for i in all_rundles_by_season[31]['rundles'] if i[0] != 'R']
all_rundles_by_season

1 leagues, 4 rundles in LL30
1 leagues, 4 rundles in LL31
1 leagues, 4 rundles in LL32
1 leagues, 4 rundles in LL33
3 leagues, 7 rundles in LL34
3 leagues, 7 rundles in LL35
3 leagues, 7 rundles in LL36
3 leagues, 7 rundles in LL37
3 leagues, 7 rundles in LL38
7 leagues, 9 rundles in LL39
7 leagues, 9 rundles in LL40
7 leagues, 9 rundles in LL41
6 leagues, 10 rundles in LL42
9 leagues, 15 rundles in LL43
9 leagues, 15 rundles in LL44
9 leagues, 15 rundles in LL45
9 leagues, 16 rundles in LL46
9 leagues, 17 rundles in LL47
10 leagues, 21 rundles in LL48
10 leagues, 21 rundles in LL49
10 leagues, 21 rundles in LL50
10 leagues, 28 rundles in LL51
10 leagues, 31 rundles in LL52
10 leagues, 34 rundles in LL53
10 leagues, 34 rundles in LL54
10 leagues, 34 rundles in LL55
13 leagues, 45 rundles in LL56
13 leagues, 53 rundles in LL57
12 leagues, 60 rundles in LL58
12 leagues, 60 rundles in LL59
8 leagues, 70 rundles in LL60
9 leagues, 82 rundles in LL61
10 leagues, 89 rundles in LL62
11 league

{30: {'leagues': ['League'],
  'rundles': [['A', 'League', 0],
   ['B', 'League', 0],
   ['C', 'League', 0],
   ['R', 'League', 0]]},
 31: {'leagues': ['League'],
  'rundles': [['A', 'League', 0], ['B', 'League', 0], ['C', 'League', 0]]},
 32: {'leagues': ['League'],
  'rundles': [['A', 'League', 0],
   ['B', 'League', 0],
   ['C', 'League', 0],
   ['R', 'League', 0]]},
 33: {'leagues': ['League'],
  'rundles': [['A', 'League', 0],
   ['B', 'League', 0],
   ['C', 'League', 0],
   ['R', 'League', 0]]},
 34: {'leagues': ['League', 'East', 'West'],
  'rundles': [['A', 'East', 0],
   ['B', 'East', 0],
   ['C', 'East', 0],
   ['A', 'West', 0],
   ['B', 'West', 0],
   ['C', 'West', 0],
   ['R', 'League', 0]]},
 35: {'leagues': ['League', 'East', 'West'],
  'rundles': [['A', 'East', 0],
   ['B', 'East', 0],
   ['C', 'East', 0],
   ['A', 'West', 0],
   ['B', 'West', 0],
   ['C', 'West', 0],
   ['R', 'League', 0]]},
 36: {'leagues': ['League', 'East', 'West'],
  'rundles': [['A', 'East', 0],
  

In [41]:
with open('rundles_by_season.json', 'w') as fp:
    json.dump(all_rundles_by_season, fp)

In [41]:
with open('run1/run1.1/player_profile_old_ids.json', 'r') as fp:
    pp = json.load(fp)

In [49]:
def rundle_md_url(rundle: list, season: int, md: int):
    zfilled_md = str(md).zfill(2)
    if season <= 38:
        return f'{constants["LLHEADER"]}/ll{season}/questions/md{zfilled_md}.shtml'
    elif season <= 51:
        url_insert = (rundle[0] if rundle[1] == 'League' else f'{rundle[0]}_{rundle[1]}') + (
            'results' if not (season == 51 and md >= 7) else '')
        return f'{constants["LLHEADER"]}/ll{season}/questions/ll{season}md{zfilled_md}{url_insert}.shtml'
    else:
        base_url = f'{constants["LLHEADER"]}/match.php?{season}&{md}&'
        if rundle[0] == 'Championship':
            return base_url + rundle[0]
        if rundle[2] == 0:
            return base_url + f'{rundle[0]}_{rundle[1]}'
        return base_url + f'{rundle[0]}_{rundle[1]}_Div_{rundle[2]}'
def format_match_result(p1_input, p2_input):
    p1_result = p1_input[0]
    p1_forfeit = p1_input[2] == 'F'
    p2_result = p2_input[0]
    p2_forfeit = p2_input[2] == 'F'
    if p1_forfeit:
        p1_result = 'F'
        p2_result = 'F' if p2_forfeit else 'W'
    elif p2_forfeit:
        p2_result = 'F'
        p1_result = 'W'
    elif p1_result == p2_result:
        p1_result = 'T'
        p2_result = 'T'
    else:
        p1_result = 'W' if int(p1_result) > int(p2_result) else 'L'
        p2_result = 'L' if p1_result == 'W' else 'W'
    return p1_result, p2_result
def pre_38(match):
    match_id = int(match['onclick'].split('/')[-1].split('.')[0])
    p1 = match.find('div', {'class': 'Nametop'})
    p1_id = p1.find('a')['href'].split('/')[-1].split('.')[0]
    # p1_id = int(sess.get(constants["LLHEADER"] + p1.find('a')['href']).url.split('?')[-1])
    p2 = match.find('div', {'class': 'Namebot'})
    p2_id = p2.find('a')['href'].split('/')[-1].split('.')[0]
    # p2_id = int(sess.get(constants["LLHEADER"] + p2.find('a')['href']).url.split('?')[-1])
    p1_input = match.find('div', {'class': 'Scoretop'}).text.strip()
    p2_input = match.find('div', {'class': 'Scorebot'}).text.strip()
    return match_id, {'id': p1_id, 'input': p1_input}, {'id': p2_id, 'input': p2_input}
def special_51(match):
    p1, p1_name, inp, p2_name, p2 = match.contents
    # p1_id = int(sess.get(constants["LLHEADER"] + p1.find('a')['href']).url.split('?')[-1])
    # p2_id = int(sess.get(constants["LLHEADER"] + p2.find('a')['href']).url.split('?')[-1])
    match_id = int(inp.find('a')['href'].split('/')[-1].split('.')[0])
    p1_id = p1.find('a')['href'].split('/')[-1].split('.')[0]
    p2_id = p2.find('a')['href'].split('/')[-1].split('.')[0]
    p1_input, p2_input = inp.text.split()
    return match_id, {'id': p1_id, 'input': p1_input}, {'id': p2_id, 'input': p2_input}
def classic_39_51(match):
    p1, p1_name, p1_input, p2_input, p2_name, p2, match_det = match.contents
    # p1_id = int(sess.get(constants["LLHEADER"] + p1.find('a')['href']).url.split('?')[-1])
    # p2_id = int(sess.get(constants["LLHEADER"] + p2.find('a')['href']).url.split('?')[-1])
    match_id = int(match_det.find('a')['href'].split('/')[-1].split('.')[0])
    p1_id = p1.find('a')['href'].split('/')[-1].split('.')[0]
    p2_id = p2.find('a')['href'].split('/')[-1].split('.')[0]
    return match_id, {'id': p1_id, 'input': p1_input.text.strip()}, {'id': p2_id, 'input': p2_input.text.strip()}
def post_52(match):
    data = match.find_all('td')
    p1_id = '?'.join(data[0].find('a')['href'].split('?')[1:])
    p2_id = '?'.join(data[4].find('a')['href'].split('?')[1:])
    match_id = int(data[2].find('a')['href'].split('?id=')[-1])
    p1_input, p2_input = data[2].text.split()
    if len(p1_input) < 3: p1_input, p2_input = False, False
    return match_id, {'id': p1_id, 'input': p1_input}, {'id': p2_id, 'input': p2_input}
def fucky(soup, p1_id, p2_id):
    full_table = soup.find('div', {'class': 'fdr'}).find('tbody').find_all('tr')
    p1_result, p2_result = '', ''
    for row in full_table:
        if 'F' in row.find('td')['class'][0]:
            tot_pts = 0
            fft = 'F'
        else:
            pts_classes = list(map(lambda q: int(q['class'][0][1]), row.find_all('td')[:6]))
            fft = list(map(lambda q: int(q.text), row.find_all('td')[:6]))
            if sum(fft) == 0:
                tot_pts = round(sum(pts_classes)*1.5)
            else:
                tot_pts = sum(np.multiply(pts_classes, fft))
        cur_id = '?'.join(row.find('a')['href'].split('?')[1:])
        if cur_id == p1_id:
            p1_result = f'{tot_pts}({fft})'
        elif cur_id == p2_id:
            p2_result = f'{tot_pts}({fft})'
    return p1_result, p2_result
def get_rundle_matchday(rundle: list, season: int, md: int,
    player_rundle_history:list=[], match_participation:list=[], pp:dict=dict()):
    url = rundle_md_url(rundle, season, md)
    gamepage = sess.get(url)
    html_text = gamepage.text
    soup = BeautifulSoup(html_text, 'html.parser')
    if season <= 38: #LL30-38
        header_text = 'Rundle ' + (rundle[0] if rundle[1] == 'League' or season <= 33 else rundle[0] + ' ' + rundle[1])
        matches = soup.find(text=header_text).parent.next_element.next_element.find_all('div', {'class': 'sbGame'})
    elif season == 51 and md >= 7: # Thorsten not at all consistent
        matches = soup.find('table', {'class': 'tblResults2'}).find_all('tr')
    elif season <= 51: #LL39-51
        matches = soup.find('table', {'class': 'tblResults'}).find_all('tr')
    else: #LL52-present
        matches = soup.find('table', {'class': 'gamelinetbl'}).find_all('tr')
    for match in matches:
        if season <= 38: match_id, p1, p2 = pre_38(match)
        elif season == 51 and md >= 7: match_id, p1, p2 = special_51(match)
        elif season <= 51: match_id, p1, p2 = classic_39_51(match)
        else: match_id, p1, p2 = post_52(match)
        if p1['input'] == False:
            p1['input'], p2['input'] = fucky(soup, str(p1['id']), str(p2['id']))
        p1_result, p2_result = format_match_result(p1['input'], p2['input'])
        if str(p1['id']).isnumeric():
            p1['id'], p2['id'] = int(p1['id']), int(p2['id'])
            p1['old_id'], p2['old_id'] = p1['id'], p2['id']
            if not p1['id'] in pp.keys():
                pp[p1['old_id']] = p1['id']
            if not p2['id'] in pp.keys():
                pp[p2['old_id']] = p2['id']
        if md == 1:
            if (season == 30 or rundle[0] == 'R'):
                if not str(p1['id']).isnumeric():
                    p1['id'], p2['id'] = '_'.join(p1['id'].split()), '_'.join(p2['id'].split())
                    if not p1['id'] in pp.keys():
                        p1['old_id'] = p1['id']
                        p1_old_url = f'/profiles/{p1["id"]}.shtml' if season <= 51 else f'/profiles.php?{p1["id"]}'
                        p1['id'] = int_or_float_or_str(sess.get(constants["LLHEADER"] + p1_old_url).url.split('?')[-1])
                        pp[p1['old_id']] = p1['id']
                    if not p2['id'] in pp.keys():
                        p2['old_id'] = p2['id']
                        p2_old_url = f'/profiles/{p2["id"]}.shtml' if season <= 51 else f'/profiles.php?{p2["id"]}'
                        p2['id'] = int_or_float_or_str(sess.get(constants["LLHEADER"] + p2_old_url).url.split('?')[-1])
                        pp[p2['old_id']] = p2['id']
            player_rundle_history.append([p1['id'], season, md, rundle[0], rundle[1], rundle[2]])
            player_rundle_history.append([p2['id'], season, md, rundle[0], rundle[1], rundle[2]])
        match_participation.append([match_id, p1['id'], p1_result])
        match_participation.append([match_id, p2['id'], p2_result])
    return player_rundle_history, match_participation, pp
get_rundle_matchday(['D', 'Magnolia', 1], 80, 21)

IndexError: string index out of range

In [75]:
player_rundle_history, match_participation, player_profile = [], [], dict()
with open('rundles_by_season.json', 'r') as fp:
    fr = json.load(fp)
full_rundles = []
# for key in fr.keys():
#     full_rundles.extend(fr[key]['rundles'])
# full_rundles
import time
st = time.time()
for season in range(30, 94):
    this_season_rundles = fr[str(season)]['rundles']
    for md in range(1, 21 if season <= 38 else 26):
        for rundle in this_season_rundles:
            try:
                player_rundle_history, match_participation, player_profile = get_rundle_matchday(
                    rundle, season, md, player_rundle_history, match_participation, player_profile
                )
            except:
                print(f'Failed on LL{season}, md {md}, rundle {rundle}, url: {rundle_md_url(rundle, season, md)}')
    print(f'completed LL{season}, total time: {time.time()-st}')
    st = time.time()
try:
    with open('match_participation.json', 'w') as a:
        json.dump(match_participation, a)
except:
    print('could not save match participation data')
try:
    with open('player_rundle_history.json', 'w') as a:
        json.dump(player_rundle_history, a)
except:
    print('could not save player rundle history data')
try:
    with open('player_profile_old_ids.json', 'w') as a:
        json.dump(player_profile, a)
except:
    print('could not save profile data')

completed LL30, total time: 63.94884777069092
completed LL31, total time: 7.904952049255371
completed LL32, total time: 16.15006685256958
completed LL33, total time: 16.884157180786133
completed LL34, total time: 32.05800104141235
completed LL35, total time: 33.9595308303833
completed LL36, total time: 34.343120098114014
completed LL37, total time: 32.774303913116455
completed LL38, total time: 37.54530692100525
completed LL39, total time: 36.277426958084106
completed LL40, total time: 33.288006067276
completed LL41, total time: 44.730040073394775
completed LL42, total time: 54.36037826538086
completed LL43, total time: 62.04588174819946
completed LL44, total time: 66.23294591903687
completed LL45, total time: 70.78969597816467
Failed on LL46, md 1, rundle ['R', 'Atlantic', 0], url: https://www.learnedleague.com/ll46/questions/ll46md01R_Atlanticresults.shtml
completed LL46, total time: 73.28983497619629
completed LL47, total time: 94.33732295036316
completed LL48, total time: 105.09880

In [50]:
with open('run3_logs.txt', 'r') as log:
    logs = log.readlines()
logs = list(filter(lambda l: 'url' in l, logs))
player_rundle_history, match_participation, player_profile = [], [], dict()
st = time.time()
for idx, log in enumerate(logs):
    spl = log.strip().split()
    url = spl[-1]
    season = int(spl[2][2:-1])
    md = int(spl[4][:-1])
    rundle = [spl[6][2], spl[7][1:-2], int(spl[8][0])]
    try:
        player_rundle_history, match_participation, player_profile = get_rundle_matchday(
            rundle, season, md, player_rundle_history, match_participation, player_profile
        )
    except:
        print(f'Failed on LL{season}, md {md}, rundle {rundle}, url: {rundle_md_url(rundle, season, md)}')
    if idx % 500 == 0:
        print(f'completed {idx-500}-{idx} in {time.time()-st}')
        st = time.time()

Failed on LL80, md 21, rundle ['D', 'Magnolia', 1], url: https://www.learnedleague.com/match.php?80&21&D_Magnolia_Div_1
completed -500-0 in 1.567363977432251


In [52]:
# manual entry for LL80 MD21 Rundle D Magnolia Div 1 bc i cant be bothered to go out of my way for it.
match_participation.extend([
    [2562712, 22580, 'T'],
    [2562712, 23499, 'T'],
    [2562737, 2074, 'W'],
    [2562737, 18616, 'L'],
    [2562762, 15422, 'L'],
    [2562762, 10874, 'W'],
    [2562787, 7166, 'W'],
    [2562787, 20319, 'L'],
    [2562812, 6850, 'W'],
    [2562812, 17285, 'L'],
    [2562837, 15966, 'W'],
    [2562837, 26651, 'F'],
    [2562862, 26642, 'T'],
    [2562862, 24475, 'T'],
    [2562887, 28657, 'L'],
    [2562887, 21690, 'W'],
    [2562912, 14087, 'L'],
    [2562912, 7305, 'W'],
    [2562937, 8134, 'L'],
    [2562937, 1197, 'W'],
    [2562962, 17412, 'W'],
    [2562962, 4766, 'F']]
)

In [53]:
len(match_participation)

286

In [54]:
with open('run1/run1.1/match_participation.json', 'r') as a:
    run1_match_participation = json.load(a)
with open('run1/run1.1/player_rundle_history.json', 'r') as b:
    run1_player_rundle_history = json.load(b)
with open('run1/run1.1/player_profile_old_ids.json', 'r') as c:
    run1_player_profile = json.load(c)
run1_player_profile

{'dages': 28118,
 'white': 19962,
 'crainc': 19713,
 'doro': 27800,
 'brooke': 20858,
 'burkland': 2803,
 'webber': 11288,
 'kerbyl': 26191,
 'fahrni': 1733,
 'petty': 1182,
 'obrien': 21905,
 'meyer': 8827,
 'roehln': 12004,
 'thomask': 18229,
 'ketteman': 3885,
 'saha': 15555,
 'chud': 27023,
 'smithm': 17927,
 'engel': 26001,
 'hewitt': 26996,
 'alvord': 11275,
 'glassmant': 19232,
 'singerd': 26550,
 'wasserman': 11100,
 'leishman': 13311,
 'hattman': 15636,
 'stanleym': 21474,
 'walkern': 24376,
 'fredricksoni': 6659,
 'lambertson': 3803,
 'wilsonc': 25988,
 'woodburyh': 27170,
 'pierson': 15979,
 'lakhaneya': 14248,
 'forshey': 14583,
 'matochak': 6018,
 'colej': 27449,
 'goldmana': 18575,
 'krugman': 22418,
 'roehld': 9847,
 'curry': 16363,
 'tuxbury': 29147,
 'north': 18078,
 'yarbro': 26204,
 'hicks': 21044,
 'zoller': 14778,
 'nathanielsz': 25578,
 'bandrowskys': 13409,
 'spykerman': 3535,
 'howardt': 14142,
 'gray': 6040,
 'ellington': 11941,
 'wedig': 29651,
 'eclipse': 166

In [55]:
print(set(player_profile.keys()).difference(set(run1_player_profile.keys())))
for key in player_profile.keys():
    run1_player_profile[key] = player_profile[key]

{36353, 34307, 35595, 35468, 36374, 36005, 35750, 35879, 35755, 35376, 33969, 36275, 34228, 34996, 35639, 35652, 35530, 36172, 35287, 34138, 34139, 35167}


In [56]:
run1_match_participation.extend(match_participation)
run1_player_rundle_history.extend(player_rundle_history)

In [57]:
with open('run1/run1.1/match_participation.json', 'w') as a:
    json.dump(run1_match_participation, a)
with open('run1/run1.1/player_rundle_history.json', 'w') as b:
    json.dump(run1_player_rundle_history, b)
with open('run1/run1.1/player_profile_old_ids.json', 'w') as c:
    json.dump(run1_player_profile, c)